In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_from_disk
# data = pd.read_csv("./data/sft/NSL-KDD-100000-sft.csv")
data = pd.read_csv("./data/sft/Mixed-sft-400000.csv")
# 划分训练集和验证集
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    data["flow"].tolist(), data["class"].tolist(), test_size=0.2, random_state=42
)
# 转换为 Hugging Face Dataset 格式
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts, "label": train_labels}),
    "eval": Dataset.from_dict({"text": eval_texts, "label": eval_labels})
})

c:\School\Anaconda\envs\LLMNIDS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 遍历数据，找到最长文本的长度（基于逗号分词）
text_lengths = [len(text.split(",")) for text in data["flow"].tolist()]
max_length = max(text_lengths)
print(f"实际设定的 max_length: {max_length}")

实际设定的 max_length: 65


In [3]:
from transformers import BertTokenizer
# 加载 BERT 分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# 逗号分词 + 重新转换为 BERT 需要的 `input_ids`
def custom_tokenize_function(examples):
    # 逗号分词
    tokenized_texts = [text.split(",") for text in examples["text"]]
    # 将每个短语转换为 BERT input_ids
    encodings = tokenizer(
        tokenized_texts,  # 逗号分词后的文本
        padding="max_length",
        max_length=max_length,
        truncation=True,
        is_split_into_words=True  # 关键参数：告诉 tokenizer 文本已经被手动分词
    )
    return encodings


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\School\Anaconda\envs\LLMNIDS\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\School\Anaconda\envs\LLMNIDS\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\School\Anaconda\envs\LLMNIDS\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\School\Anaconda\envs\LLMNIDS\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\School

In [4]:
# tokenized_datasets = dataset.map(custom_tokenize_function, batched=True)
tokenized_datasets = load_from_disk("./tokenized_datasets-100000")
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["eval"].shuffle(seed=42)

In [5]:
from transformers import AutoModelForSequenceClassification
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, hidden_dropout_prob=0.2)

c:\School\Anaconda\envs\LLMNIDS\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from peft import LoraConfig, get_peft_model
# **添加 LoRA 适配**
lora_config = LoraConfig(
    r=8,  # LoRA 低秩矩阵的秩
    lora_alpha=16,  # LoRA scaling factor
    lora_dropout=0.1,  # Dropout 防止过拟合
    target_modules=["query", "value"],  # 仅优化 Query 和 Value 层
    task_type="SEQ_CLS",  # 序列分类任务
)
# 将 LoRA 集成到 BERT 模型
lora_model = get_peft_model(base_model, lora_config)
lora_model.print_trainable_parameters()  # 查看可训练参数

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [7]:
# 继续训练模型
lora_model.train()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): L

In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
# 计算评估指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    return {
        "eval_loss": float(np.mean(logits)),  # 确保 `eval_loss` 存在
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [9]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="test_trainer-sft-diy-lora-mixed-400000_1",
    # output_dir="./trainer/test_trainer-sft-diy-lora",
    eval_strategy="epoch",
    save_strategy="epoch",  # 每个 epoch 保存一次模型
    save_steps=None,  # 取消按steps保存
    learning_rate=2e-5,  # 学习率  2e-5    5e-5
    per_device_train_batch_size=64,  # 适当增加 batch_size，默认 8
    per_device_eval_batch_size=64,
    num_train_epochs=10,  # 降低训练轮数，避免过拟合
    weight_decay=0.02,  # 加入 L2 正则化
    load_best_model_at_end=True,  # 训练结束后加载最佳模型
    logging_strategy="epoch",  # 确保每个 epoch 打印 loss
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # 早停机制
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
trainer.train()

RuntimeError: Numpy is not available